# Part and Parcel: *Predicting North American Freight Costs with Supervised Machine Learning*


### Capstone Project - Machine Learning Report
*Kimberly Kaufman*  
*August 17, 2019*
  
---

### Abstract

Despite the continuous growth of ebooks over the past decade, the publishing industry continues to fulfill a significant demand for physical, printed books. The cost of distributing print books can vary structurally based on factors such as fixed vs variable, owned vs 3PL, and other differing permutations of distribution cost models. But one significant distribution cost that is relatively constant for all publishers is the cost of freight.

At Wiley, freight costs for North American outbound volume alone can cost millions annually in USD. Although the current methods used to model these costs have been sufficient for financial planning purposes, there is most certainly room for improvement. Previous forecasts have been derived from very basic time series modeling, but we have not yet attempted to apply machine learning to improve our accuracy.

This project aims to predict the freight cost of physical book fulfillment (on both an individual shipment level and an aggregate level) by using supervised machine learning methods on a year’s worth of supply chain data out of our North American distribution center. By implementing a more complex modeling approach, we hope to improve our forecasting accuracy as well as gain better visibility into the main drivers of our costs.

For more details on this project, see the <a href="https://github.com/kaufkauf/Capstone-Project-Intermediate">GitHub repository</a>.

### Data Wrangling Recap

In [72]:
import pandas as pd
import numpy as np

# set working directory & read in files
wd = 'C:/Users/kkaufman/Documents/Data Sci/Intermediate/Freight model/'
vol1 = pd.read_csv(wd + 'FY19outboundv2 - Q1.csv', low_memory=False)
vol2 = pd.read_csv(wd + 'FY19outboundv2 - Q2.csv', low_memory=False)
vol3 = pd.read_csv(wd + 'FY19outboundv2 - Q3.csv', low_memory=False)
vol4 = pd.read_csv(wd + 'FY19outboundv2 - Q4.csv', low_memory=False)
vol = vol1.append([vol2,vol3,vol4])
missfts = pd.read_csv(wd + 'FY19missingfreight.csv', low_memory=False)
freight = pd.read_csv(wd + 'FY19freightcharges2.csv', low_memory=False) 

# new updates: fill in missing freight types
vol['SHIP_METHOD'] = vol['SHIP_METHOD'].str.strip()
vol = pd.merge(left=vol, right=missfts, how='left', left_on=['SHIP_METHOD'], right_on=['SHIP_METHOD'],copy=False)
vol['FREIGHT_TYPE'] = vol['FREIGHT_TYPE'].fillna(vol['FREIGHT TYPE'])
vol = vol.drop(columns=['STYLE CODE','FREIGHT TYPE'],axis=1)

# other data wrangling steps:
# remove trailing spaces from categoricals:
vol['SHIP_COUNTRY_NAME'] = vol['SHIP_COUNTRY_NAME'].str.title().str.strip().fillna('Unknown')
vol['FREIGHT_TYPE'] = vol['FREIGHT_TYPE'].str.strip()
# cap/floor book weight outliers at 5th/95th percentiles
vol['BOOK_WEIGHT'] = np.where(vol['BOOK_WEIGHT'] < vol['BOOK_WEIGHT'].quantile(.05), vol['BOOK_WEIGHT'].quantile(.05), vol['BOOK_WEIGHT'])
vol['BOOK_WEIGHT'] = np.where(vol['BOOK_WEIGHT'] > vol['BOOK_WEIGHT'].quantile(.95), vol['BOOK_WEIGHT'].quantile(.95), vol['BOOK_WEIGHT'])

# pivot volume data based on invoice number & convert to dataframe
idxcols = ['CJDAT1','IHINDT','IHAEDT','INVOICENO','COLLECT_METHOD','CARRIER_CODE','PRIORITY_CODE','SHIP_METHOD','FREIGHT_TYPE','SHIP_COUNTRY_NAME','MARKET_OUTLET']
valcols = ['ISBN10','TOTAL_UNITS','TOTAL_PALLETS','TOTAL_CARTONS','TOTAL_LOOSE','BOOK_WEIGHT']
dfvol = pd.pivot_table(vol, index=idxcols, columns=['GLOBAL_BUSINESS'], values=valcols, 
                       aggfunc={('ISBN10') : len,
                                ('TOTAL_UNITS') : sum,
                                ('TOTAL_PALLETS') : sum,
                                ('TOTAL_CARTONS') : sum,
                                ('TOTAL_LOOSE') : sum,
                                ('BOOK_WEIGHT') : sum}, 
                                fill_value=0)
dfvol = pd.DataFrame(dfvol)
dfvol.reset_index(inplace=True)

# remove multilevel column index & replace with concatenated column names using list comprehension
cols = [str.strip(str.replace((key + ' ' + value),'ISBN10','TOTAL_LINES')) for key, value in dfvol.columns]
dfvol.columns = dfvol.columns.get_level_values(0)
dfvol.columns = cols

# add columns for carton weight & totals
dfvol['CARTON_WEIGHT Agency'] = dfvol['BOOK_WEIGHT Agency']*dfvol['TOTAL_UNITS Agency']
dfvol['CARTON_WEIGHT Education'] = dfvol['BOOK_WEIGHT Education']*dfvol['TOTAL_UNITS Education']
dfvol['CARTON_WEIGHT Reference'] = dfvol['BOOK_WEIGHT Reference']*dfvol['TOTAL_UNITS Reference']
dfvol['CARTON_WEIGHT Test Prep'] = dfvol['BOOK_WEIGHT Test Prep']*dfvol['TOTAL_UNITS Test Prep']
dfvol['CARTON_WEIGHT Trade'] = dfvol['BOOK_WEIGHT Trade']*dfvol['TOTAL_UNITS Trade']
dfvol['TOTAL CARTONS Total'] = dfvol['TOTAL_CARTONS Agency']+dfvol['TOTAL_CARTONS Education']+dfvol['TOTAL_CARTONS Reference']+dfvol['TOTAL_CARTONS Test Prep']+dfvol['TOTAL_CARTONS Trade']
dfvol['TOTAL LOOSE Total'] = dfvol['TOTAL_LOOSE Agency']+dfvol['TOTAL_LOOSE Education']+dfvol['TOTAL_LOOSE Reference']+dfvol['TOTAL_LOOSE Test Prep']+dfvol['TOTAL_LOOSE Trade']
dfvol['TOTAL PALLETS Total']= dfvol['TOTAL_PALLETS Agency']+dfvol['TOTAL_PALLETS Education']+dfvol['TOTAL_PALLETS Reference']+dfvol['TOTAL_PALLETS Test Prep']+dfvol['TOTAL_PALLETS Trade']
dfvol['TOTAL UNITS Total'] = dfvol['TOTAL_UNITS Agency']+dfvol['TOTAL_UNITS Education']+dfvol['TOTAL_UNITS Reference']+dfvol['TOTAL_UNITS Test Prep']+dfvol['TOTAL_UNITS Trade']
dfvol['CARTON_WEIGHT Total'] = dfvol['CARTON_WEIGHT Agency']+dfvol['CARTON_WEIGHT Education']+dfvol['CARTON_WEIGHT Reference']+dfvol['CARTON_WEIGHT Test Prep']+dfvol['CARTON_WEIGHT Trade']

# join vol data to freight data
dffreight = freight[['INVNUMBER','INTDATE','TOTALCHARGE']]
df1 = pd.merge(left=dfvol, right=dffreight, how='left', left_on=['INVOICENO','CJDAT1'], right_on=['INVNUMBER','INTDATE'],copy=False)
df1['TOTALCHARGE'] = df1['TOTALCHARGE'].fillna(0)
df1 = df1.drop(columns=['INVNUMBER','INTDATE'],axis=1)

# Convert dates to dates & add monthly fields
df1['SHIP_MONTH'] = df1['IHAEDT']+19000000
df1['SHIP_MONTH'] = pd.to_datetime(df1['SHIP_MONTH'],format='%Y%m%d')
df1['SHIP_MONTH'] = df1['SHIP_MONTH'].dt.to_period('M')

# Print resulting columns
df1.head()

,CJDAT1,IHINDT,IHAEDT,INVOICENO,COLLECT_METHOD,CARRIER_CODE,PRIORITY_CODE,SHIP_METHOD,FREIGHT_TYPE,SHIP_COUNTRY_NAME,...,CARTON_WEIGHT Reference,CARTON_WEIGHT Test Prep,CARTON_WEIGHT Trade,TOTAL CARTONS Total,TOTAL LOOSE Total,TOTAL PALLETS Total,TOTAL UNITS Total,CARTON_WEIGHT Total,TOTALCHARGE,SHIP_MONTH
0,0,1180911,1180911,9477531,Y,UPSN,A3,UPS 2-Day Collect - Acct# Req.,Domestic Air Courier,United States,...,0.0,0.0,0.00,0,2,0,2,25.60,0.0,2018-09
1,1180202,1180202,1180518,6302931,Y,INTN,IX,GP OCEAN FREIGHT COLLECT,Subsidiary Ship Methods,Singapore,...,0.0,0.0,336.00,0,20,0,20,336.00,0.0,2018-05
2,1180206,1180206,1180518,6345851,Y,INTN,IX,GP OCEAN FREIGHT COLLECT,Subsidiary Ship Methods,Singapore,...,0.0,0.0,216.00,0,10,0,10,216.00,0.0,2018-05
3,1180206,1180206,1180518,6348364,Y,INTN,IX,GP OCEAN FREIGHT COLLECT,Subsidiary Ship Methods,Singapore,...,0.0,688.0,0.00,10,0,0,10,688.00,0.0,2018-05
4,1180206,1180206,1180518,6348365,Y,INTN,IX,GP OCEAN FREIGHT COLLECT,Subsidiary Ship Methods,Singapore,...,0.0,0.0,1500.24,0,5,0,19,1500.24,0.0,2018-05


In [105]:
# Additional data wrangling - adding in Region (Continent)
regions = pd.read_csv(wd + 'countries.csv', low_memory=False) 
df2 = pd.merge(left=df1, right=regions, how='left', left_on=['SHIP_COUNTRY_NAME'], right_on=['Country'],copy=False)
df2 = df2.drop(columns=['Country'],axis=1)

# Convert market outlet to categorical
#df2['MARKET_OUTLET'] = str(df2['MARKET_OUTLET'], encoding='ascii', errors='ignore')

# Remove redundant/nonrelevant columns
#df2 = df2.drop(['INVOICENO','SHIP_MONTH','SHIP_COUNTRY_NAME'],axis=1)

### Machine Learning Model

#### Preprocessing

In [106]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing as pp

# Scale continuous
scaler = pp.MinMaxScaler()
scaled1 = scaler.fit_transform(df2.iloc[:,10:51])
df2.iloc[:,10:51] = scaled1
scaled2 = scaler.fit_transform(df2.iloc[:,0:3])
df2.iloc[:,0:3] = scaled2

# Generate dummy variables
df2 = pd.DataFrame(pd.get_dummies(df2))

# Test/train split at 80% train, 20% test
X = df2.drop('TOTALCHARGE',axis=1).values
y = df2['TOTALCHARGE'].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=24)

For this dataset, we preprocessed using a min/max scaler for continuous variables as well as one-hot encoding for categorical variables.  Data was split into a training and test set using an 80/20 split, respectively.

#### Basic Linear Regression

In [109]:
# Fit & predict the model

linreg = linear_model.LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)
print('R2 =',linreg.score(X_test, y_test),', therefore 30% of the variance in the observed data can be explained by the model.')
# can also use: print(metrics.r2_score(y_test, y_pred))
print('MAE =',metrics.mean_absolute_error(y_test, y_pred))
print('MSE = ',metrics.mean_squared_error(y_test, y_pred))
print('RMSE =',np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 = 0.30688907630382656 , therefore 30% of the variance in the observed data can be explained by the model.
MAE = 4.6350903386760685
MSE =  629.9732740156463
RMSE = 25.099268396023945


In [110]:
print("Coefficient mean:",np.mean(linreg.coef_))
print("Coefficient median:",np.median(linreg.coef_))
print("Coefficient minimum:",np.min(linreg.coef_))
print("Coefficient maxmimum:",np.max(linreg.coef_))
print("")
print("Model Coefficients:")
cols = df2.drop('TOTALCHARGE',axis=1)
for idx, col_name in enumerate(cols.columns):
    print("The coefficient for {} is {}".format(col_name, linreg.coef_[0][idx]))

Coefficient mean: -1.6225903019810954
Coefficient median: -1.426133792766509
Coefficient minimum: -878.1795679223164
Coefficient maxmimum: 1569.9403827435558

Model Coefficients:
The coefficient for CJDAT1 is -1.0559740388654515
The coefficient for IHINDT is 1.429683709949395
The coefficient for IHAEDT is -323.34348965171415
The coefficient for INVOICENO is -5.8648508272085564e-09
The coefficient for MARKET_OUTLET is 6.124462822876769
The coefficient for BOOK_WEIGHT Agency is 159.45243995337103
The coefficient for BOOK_WEIGHT Education is 188.50730103458835
The coefficient for BOOK_WEIGHT Reference is 101.09642594590842
The coefficient for BOOK_WEIGHT Test Prep is -78.50115046026755
The coefficient for BOOK_WEIGHT Trade is 286.5998846172291
The coefficient for TOTAL_LINES Agency is -93.891862184638
The coefficient for TOTAL_LINES Education is -282.05650686084476
The coefficient for TOTAL_LINES Reference is 15.91609446592789
The coefficient for TOTAL_LINES Test Prep is -26.8051949143973

Given several hundred dummy variables, we can take a look ourselves at some of the coefficients for our basic categories to determine which variables might be more important to the model.  There are a lot of insights to this first round of regression training.

Our mean correlation coefficient is -1.622, whereas our median correlation coefficient is -1.42, indicating that there are a lot of extremely negative coefficients skewing our mean.  In terms of the data, this means that there are some outlier coefficients that have a negative correlation, meaning that as these variables go up, our freight cost goes down quite a bit (or, in the case of dummy variables, freight cost is much lower for the dummy variable classification than for other categorical classifications).  Given the data we're working with, this makes sense--our volume data includes a large number of shipments that aren't associated with any freight cost at all, so we can expect a lot of those cost-free shipments to have negative coefficients.

In terms of specifics, the following variable groups appear to have significant coefficients:
* Total cartons - coefficients ~ -722 for the total, and ~721-723 for each individual business
* Carton weight - coefficients ~ 173,810 for the total, and -173,810 for each individual business
* Collect method Y/N - coefficient for Y at 49, coefficient for N at -49
* Carrier code - coefficients vary
    + We can see that certain carrier codes are significant, such as CANP & CANS (Canpar/Canadian shipments) at coefficients of -35 and -48, respectively
    + FEDX, FXFE, FXNL with coefficients of 13, 97, and 149 respectively - these should be significant as I believe they indicate different types of FEDEX carriers
    + Other significant carrier codes appear to be UPSC, USPS, CGPC, CGPP
* Priority code - coefficients between -11 and 37
* Freight type, ship method
* Certain ship country names
    + This was a strange one.  Iceland appears to be the highest, at a coefficient of 330, and Namibia appears to be the lowest, at a coefficient of -129.
* Month of shipment - extremely significant correlations at ~ -100 or ~ +200

This ends up being quite a bit too many coefficients to reasonably interpret by eyeballing them, so we may want to consider some more in-depth regression models that assist with variable selection and add regularization to the mix.

#### LASSO Regression

In [114]:
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt

# Reuse train/test split from previous linear model @ 80/20 train/test

# Train and fit the model
lasso = Lasso(alpha = .001, normalize = True)
lasso.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)
print('R2 =',lasso.score(X_test, y_test))
print('MAE =',metrics.mean_absolute_error(y_test, lasso_pred))
print('MSE = ',metrics.mean_squared_error(y_test, lasso_pred))
print('RMSE =',np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)))

# Perform feature selection
print("")
lasso_coef = pd.Series(lasso.coef_, index = df2.drop('TOTALCHARGE',axis=1).columns)
print('Significant features:')
print(lasso_coef[lasso_coef > 0])

R2 = 0.28530603804714694
MAE = 4.094758716692638
MSE =  649.5902455694313
RMSE = 25.4870603555889

Significant features:
TOTAL_CARTONS Test Prep                       169.477481
TOTAL_LOOSE Test Prep                          99.278577
TOTAL_PALLETS Trade                            44.628683
TOTAL_UNITS Test Prep                         226.450734
TOTAL UNITS Total                             713.209815
COLLECT_METHOD_N                                3.871932
CARRIER_CODE_FEDX                              27.495662
CARRIER_CODE_UPSN                               1.925440
SHIP_METHOD_FEDEX FREIGHT PPD                 275.769356
SHIP_METHOD_FEDEX National Pre-Paid           531.055210
SHIP_METHOD_FedEx Int'l Priority Prepaid       82.595636
SHIP_METHOD_FedEx International Economy PP      9.714891
SHIP_METHOD_Fedex Ground Bill Recipient        28.344594
SHIP_METHOD_UPS 1-Day Air                      20.027992
SHIP_METHOD_UPS 2-Day Air                      13.136557
SHIP_METHOD_UPS Canada I

Using a Lasso regression gives us an easier idea of feature selection by dropping certain variables for us.  From the above, we can tell that collect method, total units/loose/cartons/pallets, carrier code, ship method, freight type, and ship country and region are significant variables.  We can use this to further narrow down our features and run through the model again.

In [22]:
print('Significant variables =',np.array(lasso_coef[lasso_coef > 0].index))
print(df2.shape)
print(sum(y_pred))
print(sum(y_test))

Significant variables = ['TOTAL_CARTONS Test Prep' 'TOTAL_LOOSE Test Prep' 'TOTAL_PALLETS Trade'
 'TOTAL_UNITS Test Prep' 'TOTAL UNITS Total' 'COLLECT_METHOD_N'
 'CARRIER_CODE_FEDX' 'CARRIER_CODE_UPSN' 'SHIP_METHOD_FEDEX FREIGHT PPD'
 'SHIP_METHOD_FEDEX National Pre-Paid'
 "SHIP_METHOD_FedEx Int'l Priority Prepaid"
 'SHIP_METHOD_FedEx International Economy PP'
 'SHIP_METHOD_Fedex Ground Bill Recipient' 'SHIP_METHOD_UPS 1-Day Air'
 'SHIP_METHOD_UPS 2-Day Air' "SHIP_METHOD_UPS Canada Int'l Saver Prepaid"
 'SHIP_METHOD_UPS Freight Pre-Paid' 'SHIP_METHOD_UPS Ground'
 'SHIP_METHOD_UPS Next Day Sat. Delivery'
 'SHIP_METHOD_UPS Puerto Rico 1-Day' 'SHIP_METHOD_UPS Puerto Rico 2-Day'
 'FREIGHT_TYPE_Domestic Air Courier' 'SHIP_COUNTRY_NAME_Costa Rica'
 'SHIP_COUNTRY_NAME_Guatemala' 'SHIP_COUNTRY_NAME_Iceland'
 'SHIP_COUNTRY_NAME_Panama' 'SHIP_COUNTRY_NAME_Peru'
 'SHIP_COUNTRY_NAME_Puerto Rico']
(419852, 417)
[550982.54304443]
[541640.98600003]


#### Ridge Regression

In [120]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha = 0.1, normalize = True)
ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)

print('R2 =',ridge.score(X_test, y_test))
print('MAE =',metrics.mean_absolute_error(y_test, ridge_pred))
print('MSE = ',metrics.mean_squared_error(y_test, ridge_pred))
print('RMSE =',np.sqrt(metrics.mean_squared_error(y_test, ridge_pred)))

R2 = 0.276617126410046
MAE = 4.40365361211475
MSE =  657.4876569714431
RMSE = 25.64152212664925


We can see from the previous three examples that there isn't a huge amount of variation between basic, LASSO, and ridge regression, but our best RMSE definitely comes from the basic linear model.  It is interesting to see, however, that our best mean absolute error comes from the LASSO model.  Next, we'll do some feature selection and see how well we can improve our RMSE for this model.

#### Finalized Linear Regression

In [136]:
# Preprocessing
df3 = df1

# Scale continuous
scaler = pp.MinMaxScaler()
scaled1 = scaler.fit_transform(df2.iloc[:,10:51])
df3.iloc[:,10:51] = scaled1
scaled2 = scaler.fit_transform(df2.iloc[:,0:3])
df3.iloc[:,0:3] = scaled2

# Remove irrelevant variables
df3 = df3.drop(['CJDAT1','IHINDT','IHAEDT','INVOICENO','MARKET_OUTLET','CARRIER_CODE','PRIORITY_CODE','FREIGHT_TYPE'], axis=1)

# Generate dummy variables
df3 = pd.DataFrame(pd.get_dummies(df3))

# Test/train split at 80% train, 20% test
X2 = df3.drop('TOTALCHARGE',axis=1).values
y2 = df3['TOTALCHARGE'].values.reshape(-1,1)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2, random_state=24)

# Fit & predict the model
linreg = linear_model.LinearRegression()
linreg.fit(X2_train, y2_train)
y2_pred = linreg.predict(X2_test)
print('R2 =',linreg.score(X2_test, y2_test),', which represents the % of the variance in the observed data can be explained by the model.')
# can also use: print(metrics.r2_score(y_test, y_pred))
print('MAE =',metrics.mean_absolute_error(y2_test, y2_pred))
print('MSE = ',metrics.mean_squared_error(y2_test, y2_pred))
print('RMSE =',np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))

R2 = 0.9998176662458134 , which represents the % of the variance in the observed data can be explained by the model.
MAE = 0.0022092665570426365
MSE =  0.16572440018109752
RMSE = 0.4070926186767546


In [134]:
print(np.sum(y2_test))
print(np.sum(y2_pred))

541640.986
541508.1954696729


In this case, we removed all of the date fields since all but the shipment date (IHAEDT) are essentially meaningless for this purpose, and we're already capturing shipment date in the ship month, which had a much higher coefficient originally than the exact dates.  We also removed market outlet, which never had a very high coefficient, and all freight-related variables *except* for ship method, which had some particularly high coefficients.  So that means freight type, carrier, and priority were removed.


**Notes:  
Remove all original variables (dates, invoice nos, market outlet & all freight vars except ship method):**  
R2 = 0.9998176662458134 , which represents the % of the variance in the observed data can be explained by the model.  
MAE = 0.0022092665570426365  
MSE =  0.16572440018109752  
RMSE = 0.4070926186767546  

**Add back in freight type, for example:**  
R2 = 0.9336041702554276 , which represents the % of the variance in the observed data can be explained by the model.  
MAE = 0.046193192539711055  
MSE =  60.34762519990086  
RMSE = 7.768373394726908  

In [137]:
print(df3.columns)

Index(['BOOK_WEIGHT Agency', 'BOOK_WEIGHT Education', 'BOOK_WEIGHT Reference',
       'BOOK_WEIGHT Test Prep', 'BOOK_WEIGHT Trade', 'TOTAL_LINES Agency',
       'TOTAL_LINES Education', 'TOTAL_LINES Reference',
       'TOTAL_LINES Test Prep', 'TOTAL_LINES Trade',
       ...
       'SHIP_MONTH_2018-07', 'SHIP_MONTH_2018-08', 'SHIP_MONTH_2018-09',
       'SHIP_MONTH_2018-10', 'SHIP_MONTH_2018-11', 'SHIP_MONTH_2018-12',
       'SHIP_MONTH_2019-01', 'SHIP_MONTH_2019-02', 'SHIP_MONTH_2019-03',
       'SHIP_MONTH_2019-04'],
      dtype='object', length=325)
